In [20]:
import sys
import shutil
sys.path.append('../src/')
from helpers import finmag
import textwrap 
import os
import sys
import os; import dolfin as df; import gc; import sys;
import pickle; import numpy as np; from finmag import Simulation as Sim
from finmag.energies import Exchange, DMI, Demag, Zeeman, UniaxialAnisotropy
from dolfinh5tools import openh5


def load_h5_field(sim, filename):
    h5file = df.HDF5File(sim.mesh.mpi_comm(), filename, 'r')
    field = df.Function(sim.S3)
    h5file.read(field, 'm')
    h5file.close()
    return field

def save_h5_field(field, mesh, filename, fieldname='m'):
    h5file = df.HDF5File(mesh.mpi_comm(), filename, 'w')
    h5file.write(mesh, 'mesh')
    h5file.write(field, fieldname)
    del h5file


def load_m_h5(sim, filename):
    print("\n\nLoading {} to field\n\n".format(filename))
    h5file = df.HDF5File(sim.mesh.mpi_comm(), filename, 'r')
    field = df.Function(sim.S3)
    h5file.read(field, 'm')
    h5file.close()
    return field

def schedule_m_h5(sim, filename, inc=[0], **kwargs):
    print(kwargs)
    f = filename.split('.')
    filen = '.'.join(f[:-1])
    filename = "{}_{:06}.h5".format(filen, inc[0])
    print('Saving Field to h5 file {}'.format(filename))
    save_h5_field(sim.llg._m_field.f, sim.mesh, filename)
    print('Saved Field to h5 file (Snapshot #{})'.format(inc[0]))
    inc[0] += 1


def disk_with_internal_layers(d, layer_positions, lmax, name=''):
    """Creates a disk mesh with a flat interface inside.
    Args:
        d - disk diameter
        layer_positions - assuming layer 0 is at height 0, layer 1 is at layer_positions[0], etc...
        lmax - discretisation

    """
    # gmsh geometry script (obtained using gmsh)
    geo_script = textwrap.dedent("""\
    lmax = DefineNumber[ $lmax$, Name "Parameters/lmax" ];
    rad = DefineNumber[ $rad$, Name "Parameters/rad" ];
    Point(1) = {0, 0, 0, lmax};
    Point(2) = {rad, 0, 0, lmax};
    Point(3) = {-rad, 0, 0, lmax};
    Point(4) = {0, rad, 0, lmax};
    Point(5) = {0, -rad, 0, lmax};
    Circle(1) = {4, 1, 2};
    Circle(2) = {2, 1, 5};
    Circle(3) = {5, 1, 3};
    Circle(4) = {3, 1, 4};
    Line Loop(5) = {4, 1, 2, 3};
    Ruled Surface(6) = {5};
    """)
    for i, l in enumerate(layer_positions):
        if i == 0:
            geo_script += textwrap.dedent("""\
            
                out1[] = Extrude {{0, 0, {}}} {{
                    Surface{{6}};
                }};

            """).format(l)
        else:
            geo_script += textwrap.dedent("""\
            
                out{}[] = Extrude {{0, 0, {}}} {{
                    Surface{{out{}[0]}};
                }};
                
            """).format(i+1, l-layer_positions[i-1], i)

    # Replace parameters in the gmsh geometry script.
    geo_script = geo_script.replace('$rad$', str(d/2.))
    geo_script = geo_script.replace('$lmax$', str(lmax))

    #print(geo_script)
    
    # Write the geometry script to the .geo file.
    # basename = 'disk_with_boundary-{}-{}-{}-{}-{}'.format(name, str(d/2.0), jid, aid, '_'.join(layers))
    basename = 'disk_with_boundary-{}-{}'.format(name, str(d/2.0))
    print('\n\nMESH FILENAMES = {}\n\n'.format(basename))
    geo_file = open(basename + '.geo', 'w')
    geo_file.write(geo_script)
    geo_file.close()

    # Create a 3d mesh.
    gmsh_command = 'gmsh {}.geo -3 -optimize_netgen -o {}.msh'.format(basename, basename)
    os.system(gmsh_command)

    # Convert msh mesh format to the xml (required by dolfin).
    dc_command = 'dolfin-convert {}.msh {}.xml'.format(basename, basename)
    os.system(dc_command)

    # Load the mesh and create a dolfin object.
    mesh = df.Mesh('{}.xml'.format(basename))

    # Delete all temporary files.
    # os.system('rm {}.geo {}.msh {}.xml'.format(basename, basename, basename))

    return mesh


L = 70.0
top_and_bottom_height = 0
middle_height = L
diameter=3*L

#layers = [top_and_bottom_height,
#          top_and_bottom_height+middle_height,
#          2*top_and_bottom_height+middle_height]

layers=[L]
path = '/home/rp20g15/hopfion-fege/submission-scripts/test_{}'.format(L)
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)
os.chdir(path)




mesh = disk_with_internal_layers(d=diameter, layer_positions=layers, lmax=3)


print(finmag.util.meshes.mesh_quality(mesh))

def K_init(K, layers):
    def wrapped_function(pos):
        x, y, z = pos
        # The mesh was created so that the boundary between grains is
        # at z=0.
        if z <= layers[0]:
            return K
        elif z > layers[0] and z <= layers[1]:
            return 0
        else:
            return K
    return wrapped_function



Ms = 3.84e5  # magnetisation saturation (A/m)
A = 8.78e-12  # exchange energy constant (J/m)
D = 1.58e-3  # DMI constant (J/m**2)

#K = 100*(4*np.pi*1e-7)*Ms**2
#print('K = {}'.format(K))

# initial magnetisation configuration


def m_init(pos):
    x, y, z = pos
    rho = (x**2 + y**2)**0.5
    theta = np.arctan2(y, x)
    if z > top_and_bottom_height + df.DOLFIN_EPS and z < middle_height+top_and_bottom_height - df.DOLFIN_EPS:
         mx = np.sin(2.0*np.pi*rho/L)*np.sin(theta)
         my = -np.sin(2.0*np.pi*rho/L)*np.cos(theta)
         mz = np.cos(2.0*np.pi*rho/L)
         return mx, my, mz
    else:
        return (0, 0, 1)


def m_init(pos):
    """
    Initial state corresponding to Hopf ansatz mapped to a cylinder;
    see Paul Sutcliffe's Hopfion paper for details
    """
    x, y, z = pos
    # Shift coordinates in the z-direction as our mesh is not origin centred.
    z -= L/2.0
    rho = (x**2 + y**2)**0.5
    theta = np.arctan2(y, x)
    omega = np.tan(np.pi*z/L)
    sigma = (1 + (2*z/L)**2)*(1/np.cos(np.pi*rho/(2*L)))/L
    gamma = sigma**2 * rho ** 2 + omega**2 / 4.0
    mx = 4*sigma*rho * (omega*np.cos(theta) - (gamma - 1)*np.sin(theta)) / (1 + gamma)**2
    my = 4*sigma*rho * (omega * np.sin(theta) + (gamma-1)*np.cos(theta)) / (1 + gamma)**2
    mz = 1 - (8 * sigma**2 * rho**2) / (1 + gamma)**2
    return mx, my, mz

# name = "cylinder_rad_{}_h_{}".format(diameter,
#                                             top_and_bottom_height,
#                                             middle_height)

name = 'cylinder'
sim = Sim(mesh, Ms, unit_length=1e-9, name=name)


# Add energies. No Zeeman because the system is in zero field.
sim.add(Exchange(A))
sim.add(DMI(D))
sim.add(Demag())
#sim.add(UniaxialAnisotropy(K_init(K, layers=layers), (0, 0, 1)))

def borders_pinning(pos):
    x, y, z = pos[0], pos[1], pos[2]
    if (z < df.DOLFIN_EPS) or (z >  L*0.99):
        return True
    else:
        return False

# Pass the function to the 'pins' property of the simulation
sim.pins = borders_pinning

# Turn off precession.
sim.do_precession = False

# Initialise the system.
sim.set_m(m_init)

save_h5_field(sim.llg._m_field.f, sim.mesh, 'initial_m.h5')

sim.save_vtk('initial.pvd', overwrite=True)
#sim.schedule('save_vtk', every=5e-12, filename='m.pvd')
sim.schedule(schedule_m_h5, every=5e-12, filename='m.h5', overwrite=True)
# relax with high precision
sim.relax(stopping_dmdt=1e-6, save_vtk_snapshot_as='m_relax.pvd')






MESH FILENAMES = disk_with_boundary--105.0




[2018-12-14 09:41:37] INFO: Finmag logging output will be written to file: '/home/rp20g15/hopfion-fege/submission-scripts/test_70.0/cylinder.log' (any old content will be overwritten).
[2018-12-14 09:41:37] DEBUG: Creating DataWriter for file 'cylinder.ndt'
[2018-12-14 09:41:37] INFO: Creating Sim object name='cylinder', instance_id=10 (rank=0/1).
[2018-12-14 09:41:37] DEBUG:    Total number of Sim objects in this session: 11
[2018-12-14 09:41:37] INFO: <Mesh of topological dimension 3 (tetrahedra) with 53404 vertices and 270406 cells, ordered>


======== Mesh quality info: ========
0.192 
0.233 
0.273 
0.313 
0.354 
0.394 
0.434 
0.475 
0.515 
0.555 *
0.596 **
0.636 ***
0.677 ******
0.717 *********
0.757 ****************
0.798 *************************
0.838 ***************************************
0.878 ********************************************************
0.919 **********************************************************************
0.959 ***************************************************************
0.999 ***********************



[2018-12-14 09:41:37] DEBUG: Creating LLG object.
[2018-12-14 09:41:40] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2018-12-14 09:41:40] DEBUG: Adding interaction Exchange to simulation.
[2018-12-14 09:41:43] DEBUG: Creating DMI object with method box-matrix-petsc, in Jacobian.
[2018-12-14 09:41:43] DEBUG: Adding interaction DMI to simulation.
[2018-12-14 09:41:46] DEBUG: Creating Demag object with solver 'FK'.
[2018-12-14 09:41:46] DEBUG: Adding interaction Demag to simulation.
[2018-12-14 09:41:47] DEBUG: Using Krylov solver for demag.
[2018-12-14 09:41:50] DEBUG: Boundary element matrix uses 1740.27 MB of memory.
[2018-12-14 09:42:21] INFO: Creating integrator with backend sundials and arguments {'reltol': 1e-06, 'abstol': 1e-06}.
[2018-12-14 09:42:21] DEBUG: Updating get method for steps in TableWriter(name=cylinder.ndt)
[2018-12-14 09:42:21] DEBUG: Updating get method for last_step_dt in TableWriter(name=cylinder.ndt)
[2018-12-14 09:42:21] DEBUG: Up

{'overwrite': True}
Saving Field to h5 file m_000000.h5
Saved Field to h5 file (Snapshot #0)


...

[2018-12-14 10:24:56] DEBUG: At t=1.32e-08, last_dmdt=3.21e+03 * stopping_dmdt, next dt=1e-10.


{'overwrite': True}
Saving Field to h5 file m_002645.h5
Saved Field to h5 file (Snapshot #2645)
{'overwrite': True}
Saving Field to h5 file m_002646.h5
Saved Field to h5 file (Snapshot #2646)
{'overwrite': True}
Saving Field to h5 file m_002647.h5
Saved Field to h5 file (Snapshot #2647)
{'overwrite': True}
Saving Field to h5 file m_002648.h5
Saved Field to h5 file (Snapshot #2648)
{'overwrite': True}
Saving Field to h5 file m_002649.h5
Saved Field to h5 file (Snapshot #2649)
{'overwrite': True}
Saving Field to h5 file m_002650.h5
Saved Field to h5 file (Snapshot #2650)
{'overwrite': True}
Saving Field to h5 file m_002651.h5
Saved Field to h5 file (Snapshot #2651)
{'overwrite': True}
Saving Field to h5 file m_002652.h5
Saved Field to h5 file (Snapshot #2652)
{'overwrite': True}
Saving Field to h5 file m_002653.h5
Saved Field to h5 file (Snapshot #2653)
{'overwrite': True}
Saving Field to h5 file m_002654.h5
Saved Field to h5 file (Snapshot #2654)
{'overwrite': True}
Saving Field to h5 f

[2018-12-14 10:24:59] DEBUG: At t=1.33e-08, last_dmdt=3.21e+03 * stopping_dmdt, next dt=1e-10.
[2018-12-14 10:24:59] DEBUG: dmdt 1.0 times larger than last time (counting 9/10).


{'overwrite': True}
Saving Field to h5 file m_002665.h5
Saved Field to h5 file (Snapshot #2665)
{'overwrite': True}
Saving Field to h5 file m_002666.h5
Saved Field to h5 file (Snapshot #2666)
{'overwrite': True}
Saving Field to h5 file m_002667.h5
Saved Field to h5 file (Snapshot #2667)
{'overwrite': True}
Saving Field to h5 file m_002668.h5
Saved Field to h5 file (Snapshot #2668)
{'overwrite': True}
Saving Field to h5 file m_002669.h5
Saved Field to h5 file (Snapshot #2669)
{'overwrite': True}
Saving Field to h5 file m_002670.h5
Saved Field to h5 file (Snapshot #2670)
{'overwrite': True}
Saving Field to h5 file m_002671.h5
Saved Field to h5 file (Snapshot #2671)
{'overwrite': True}
Saving Field to h5 file m_002672.h5
Saved Field to h5 file (Snapshot #2672)
{'overwrite': True}
Saving Field to h5 file m_002673.h5
Saved Field to h5 file (Snapshot #2673)
{'overwrite': True}
Saving Field to h5 file m_002674.h5
Saved Field to h5 file (Snapshot #2674)
{'overwrite': True}
Saving Field to h5 f

[2018-12-14 10:25:01] DEBUG: At t=1.34e-08, last_dmdt=3.21e+03 * stopping_dmdt, next dt=1e-10.
[2018-12-14 10:25:01] DEBUG: dmdt 1.0 times larger than last time (counting 10/10).
[2018-12-14 10:25:01] WARNING: Stopping time integration after dmdt increased 10 times without a decrease in energy (which indicates that something might be wrong).
[2018-12-14 10:25:01] DEBUG: Re-initialising CVODE integrator.
[2018-12-14 10:25:01] DEBUG: Re-initialising CVODE integrator.
[2018-12-14 10:25:01] INFO: Relaxation finished at time t = 1.3e-08.
[2018-12-14 10:25:02] DEBUG: Saved field at t=1.34244548293e-08 to file 'm_relax.pvd' (snapshot #0; saving took 1.34 seconds).


In [21]:
save_h5_field(sim.llg._m_field.f, sim.mesh, 'relaxed.h5')

In [22]:
n_eigenmodes = 70 # Calculate the first n_eigenmodes.
complex_freqs = sim.compute_normal_modes(n_values=n_eigenmodes)
f_array = np.real(complex_freqs[0])

np.save('frequencies.npy', f_array)

sim.export_eigenmode_animations(n_eigenmodes, directory='/scratch/rp20g15/eigenmode-anims/hopfion-fege-L_70',
                                num_cycles=1,
                                num_snapshots_per_cycle=30,
                                save_h5=True)


AttributeError: 'Simulation' object has no attribute 'compute_normal_modes'